In [1]:
from hypersurface_tf import *
from generate_h import *
from complexNN import *
import sympy as sp
import tensorflow as tf

### Prepare the dataset:

In [2]:
z0, z1, z2, z3, z4 = sp.symbols('z0, z1, z2, z3, z4')
Z = [z0,z1,z2,z3,z4]
f = z0**5 + z1**5 + z2**5 + z3**5 + z4**5 + 0.5*z0*z1*z2*z3*z4
HS = Hypersurface(Z, f, 10000)

In [3]:
HS.set_k(2)

In [4]:
patch = HS.patches[0].patches[0]

In [5]:
x = tf.convert_to_tensor(np.array(patch.points, dtype=np.complex64))

In [6]:
y = tf.cast(patch.num_eta_tf('FS'), dtype=tf.complex64)

In [7]:
dataset = tf.data.Dataset.from_tensor_slices((x, y))

In [8]:
dataset = dataset.batch(10000)

### Build the model:

In [12]:
class KahlerMetric(tf.keras.Model):

    def __init__(self):
        super(KahlerMetric, self).__init__()
        self.layer_1 = ComplexDense(5, 10, activation=tf.square)
        self.layer_2 = ComplexDense(10, 10, activation=tf.square)
        self.g = ComplexDense(10, 10)
        
    def call(self, inputs):
        with tf.GradientTape(watch_accessed_variables=False, persistent=True) as t:
            t.watch(inputs)
            with tf.GradientTape(watch_accessed_variables=False) as tt:
                tt.watch(inputs)
                x = self.layer_1(inputs)
                x = self.layer_2(x)
                x = self.g(x)
                x = tf.linalg.trace(tf.matmul(x, x, adjoint_b=True))
                x = tf.math.log(x)
                dk_dzbar = tt.gradient(x, inputs)
            kahler_metric = t.jacobian(dk_dzbar, inputs)
            #kahler_metric = t.jacobian(dk_dzbar, inputs) + tf.math.conj(t.jacobian(-dk_dzbar*1j, inputs))*1j
        return kahler_metric

In [13]:
model = KahlerMetric()

In [14]:
for step, (x_batch_train, y_batch_train) in enumerate(dataset):
    with tf.GradientTape() as tape:
        kahler_metric = model(x_batch_train)
        print(kahler_metric)
        #volume_form = tf.matmul(patch.r_tf, tf.matmul(kahler_metric, patch.r_tf, adjoint_b=True))

ResourceExhaustedError: OOM when allocating tensor with shape[2410,10] and type complex64 on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Conj]